# [SETUP] 
connect to DuckDB

In [ ]:
# Load the extension
%load_ext sql

In [ ]:
# Connect to DuckDB
%sql duckdb:///../../tpch.db

In [ ]:
%config SqlMagic.displaylimit = 100

In [ ]:
%%sql
-- Run a simple show tables
SELECT
  table_name
FROM
  information_schema.tables
WHERE
  table_schema = 'main'

In [ ]:
# If you do not see any tables run the below command (after uncommeting it)
#! python setup.py

# [HOW] to find n-th event in a series of events with Window functions

* Many user interactions are stored as events (e.g., impressions, clicks, checkouts, cab called, cab boarded, cab dismounted, etc.)

* Analytical questions involve identifying one or more of such events and associating it with a past event. 

* For example, if a customer purchases a product, how did the user land on the product page (google, ads, Bing, etc.) (aka attribution)?

[ref: utm](https://blog.hubspot.com/customers/understanding-basics-utm-parameters)



## [Example] Find n-th click in a series of user clicks 

* Assume we have a `clickstream` table with user_id and the time they clicked on our web page. We can use ranking functions to pick the user's 3rd (or any n-th) click.

* n-th event is a series of events that is beneficial in
	* Marketing attribution
	* Debugging issues with late-arriving data



For example, let's find the 3rd click in a series of clicks:

![3-rd click](../../images/3click.png)

In [ ]:
%%sql
WITH clickstream AS (
    SELECT
        1 AS user_id, '2024-07-01 10:00:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:05:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:15:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:25:00' AS click_time
),
ranked_clicks AS (
    SELECT
        user_id,
        click_time,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY click_time) AS click_rank
    FROM
        clickstream
)
SELECT
    user_id,
    click_time,
    click_rank
FROM
    ranked_clicks
WHERE
    click_rank = 3;


* This pattern(ROW_NUMBER + ORDER BY unique key) can also remove duplicate rows. 

* Note: some DBS support drop duplicate function

Let's see how we can drop duplicates with this approach:

![Remove duplicates](../../images/dupclick.png)

## [Exercise] 

Write a query to remove duplicates from the clickstream data (created as CTE below)

**Time limit during live workshop: 5 min**

**Hint:**
  1. Think about how you can use `row_number` as shown above to remove duplicates.

In [ ]:
%%sql
WITH clickstream AS (
    SELECT
        1 AS user_id, '2024-07-01 10:00:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:05:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        1 AS user_id, '2024-07-01 10:10:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:15:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:20:00' AS click_time UNION ALL
    SELECT
        2 AS user_id, '2024-07-01 10:25:00' AS click_time
),
-- your code here

# [HOW] Converting row values into individual columns (aka PIVOT)

* Commonly used for easy visual summarization

* Used extensively by business folks to inspect value distributions

Let's take a look at how PIVOT works: 

Consider that you have this table,

| order_month | o_orderpriority    | o_totalprice |
|-------------|--------------------|--------------|
| 1996-01     | 5-LOW              | 173665.47    |
| 1996-12     | 1-URGENT           | 46929.18     |
| 1993-10     | 5-LOW              | 193846.25    |
| 1995-10     | 5-LOW              | 32151.78     |
| 1994-07     | 5-LOW              | 144659.2     |
| 1992-02     | 4-NOT SPECIFIED    | 58749.59     |
| 1996-01     | 2-HIGH             | 252004.18    |
| 1995-07     | 2-HIGH             | 208660.75    |
| 1993-10     | 3-MEDIUM           | 163243.98    |
| 1998-07     | 3-MEDIUM           | 58949.67     |


Pivoting it by `o_orderpriority` will convert the values in this column into individual columns as shown below:

| order_month | 1-URGENT  | 2-HIGH     | 3-MEDIUM   | 4-NOT SPECIFIED | 5-LOW      |
|-------------|-----------|------------|------------|-----------------|------------|
| 1992-02     | &nbsp;    | &nbsp;     | &nbsp;     | 58749.59        | &nbsp;     |
| 1993-10     | &nbsp;    | &nbsp;     | 163243.98  | &nbsp;          | 193846.25  |
| 1994-07     | &nbsp;    | &nbsp;     | &nbsp;     | &nbsp;          | 144659.2   |
| 1995-07     | &nbsp;    | 208660.75  | &nbsp;     | &nbsp;          | &nbsp;     |
| 1995-10     | &nbsp;    | &nbsp;     | &nbsp;     | &nbsp;          | 32151.78   |
| 1996-01     | &nbsp;    | 252004.18  | &nbsp;     | &nbsp;          | 173665.47  |
| 1996-12     | 46929.18  | &nbsp;     | &nbsp;     | &nbsp;          | &nbsp;     |
| 1998-07     | &nbsp;    | &nbsp;     | 58949.67   | &nbsp;          | &nbsp;     |


`PIVOT` is a very common use case for business people using excel. It is typically used to visually inspect distribution of data (aka do the numbers align with business knowledge).

![Pivot in Excel](../../images/pivot.png)

## Use GROUP BY + CASE WHEN (conditional aggregate) to replicate PIVOT in SQL

* Pivots take values in rows and convert them into columns.

* We can create this logic in SQL with a CASE WHEN inside a GROUP BY 

* Only columns with a low number of unique values (aka low cardinality) are pivoted.

* Convert `orderpriority` column values into individual columns and calculate monthly revenue.


## [Example]

In [ ]:
%%sql
SELECT strftime(o_orderdate, '%Y-%m') AS ordermonth,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '1-URGENT' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS urgent_order_avg_price,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '2-HIGH' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS high_order_avg_price,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '3-MEDIUM' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS medium_order_avg_price,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '4-NOT SPECIFIED' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS not_specified_order_avg_price,
       ROUND(AVG(CASE
                     WHEN o_orderpriority = '5-LOW' THEN o_totalprice
                     ELSE NULL
                 END), 2) AS low_order_avg_price
FROM orders
GROUP BY strftime(o_orderdate, '%Y-%m');



## [Exercise] 

Write a query to pivot orders data by orderpriority and show average total price grouped by year
    
**Time limit during live workshop: 5 min**

**Hint**: 
    1. Use `strftime(o_orderdate, '%Y') AS order_year` to get order_year.

`orders` table schema: ![Orders](../../images/orders.png)

In [ ]:
%%sql
-- your query here

Some DBs support PIVOT


In [ ]:
%%sql
-- Doesn't work on Jupyter
PIVOT
  (SELECT *,
          strftime(o_orderdate, '%Y-%m') AS order_month
   FROM orders) ON o_orderpriority USING AVG(o_totalprice)
GROUP BY order_month;

# [HOW] Most analytical dashboards need period-over-period comparison

* Take a look at these popular analytical websites. You will see a few key numbers in big fonts next to a smaller `+/-number` indicating the change percentage.

* People are interested in seeing how performance has changed over time

* Dashboards show metrics for a certain period and often show how they have changed compared to the prior period.

Let's look at some examples:

![](../../images/dash.png)

This is [Startdataengineering](https://www.startdataengineering.com/)'s user analytics via [Plausible](https://plausible.io/):

![Startdataengineering analytics](../../images/pop.png)


## Use group by to create metrics at the chosen period grain and window value functions to compare the current period with the previous period

* Write a query on the `orders` table that has the following output:
	1. ordermonth (in YYYY-MM format)
	2. Revenue: Sum of totalprice for that month
	3. revenue_MOM_change: The current month's revenue - the previous month's revenue



## [Example]

In [ ]:
%%sql
SELECT order_month,
       revenue,
       revenue - lag(revenue) OVER (
                                    ORDER BY order_month) AS revenue_MOM_change,
       ROUND((revenue - lag(revenue) OVER (
                                           ORDER BY order_month)) / revenue, 2) AS perc_revenue_MOM_change
FROM
  (SELECT strftime(o_orderdate, '%Y-%m') AS order_month,
          SUM(o_totalprice) AS revenue
   FROM orders
   GROUP BY 1)
ORDER BY 1 ;


## [Exercise]


**Time limit during live workshop: 10 min**

* Scenario: You are designing a data set for a dashboard. The dashboard should be able to show metrics at day, week, month, and year levels (assume these are drop-downs on the dashboard).

* Assume that you, the data engineer assigned to building the table necessary for the dashboard.

* Question 1: What clarifying questions would you ask the dashboard team?

* Question 2: How would you design the table to be used by the dashboard software? What are the considerations you need to be mindful of?



# [Things to note] with Window data access patterns

* Query patterns: qps, other filters

* Performance consideration: Size of granular data

* Pre aggregation: data freshness, additive, nonadditive metrics


# Recap

* Find the nth event in a series of events with ranking window functions

* Do pivot in SQL with a CASE WHEN inside an aggregated function

* Do period-over-period change calculation with value (lead/lag) window function

* Window functions are expensive; if your use case requires repeated use of window functions, consider pre-aggregating your data



# Helpers

1. Solutions are available at [workshop_solutions](./workshop_solutions.ipynb). **Note** You need to stop the kernel in this notebook before starting the next one, since DuckDB can only have one worker on it at a time.
2. Note the `outline`(or `Table of Contents` in the left pane on Jupyter notebook) is a easy way to navigate this workbook.

# Questions